# 5. Segregation Indexes

## Prerequistes

In [5]:
import numpy as np
import pandas as pd
import pickle
import graph_tool as gt
from graph_tool.spectral import adjacency
import scipy.sparse as sp

In [31]:
# Load Types Matrix
data = np.load('/mnt/disk2/Data/Matrices/Useful_Matrices/types_matrix.npz')
indices = data['indices']
indptr = data['indptr']
shape = data['shape']
data = data['data']
types = sp.csr_matrix((data, indices, indptr), shape = shape)

# Load Types Matrix
data = np.load('/mnt/disk2/Data/Matrices/3_Day_Window/adj_end_of_30-04-2021.csr.npz')
indices = data['indices']
indptr = data['indptr']
shape = data['shape']
data = data['data']
adj = sp.csr_matrix((data, indices, indptr), shape = shape)

In [38]:
# Types Matrix is a crs matrix. The types of the matrix is as follows: Derecha, Izquierda, Centro, No Retweet
types

Matriz de Tipos: 


<37337x4 sparse matrix of type '<class 'numpy.int32'>'
	with 64542 stored elements in Compressed Sparse Row format>

In [34]:
adj

<37337x37337 sparse matrix of type '<class 'numpy.float64'>'
	with 924709 stored elements in Compressed Sparse Row format>

In [39]:
# Contact Layer. Corresponds to the number of relation between and within groups acoording to the groups indexes
M = types.transpose().dot(adj).dot(types)

M.toarray()

array([[ 313958.,  304581.,  314412.,   53308.],
       [ 304581., 1113976., 1019961.,   79963.],
       [ 314412., 1019961.,  949182.,   82660.],
       [  53308.,   79963.,   82660.,   11668.]])